In [ ]:
# Data (pre-)processing
import os
import pandas as pd
import numpy as np
from fcsy import DataFrame
from sklearn.utils import shuffle

# Modeling
import hdbscan
import umap
import joblib
from sklearn.kernel_approximation import Nystroem
from sklearn.linear_model import SGDOneClassSVM
from sklearn.pipeline import make_pipeline
from sklearn.neighbors import LocalOutlierFactor

# Performance
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score
pd.set_option('mode.chained_assignment', None)

In [ ]:
# INPUT FOLDERS
# Path of preprocessed LAIP29 FCS files
LAIP29_FCS_path = "data/LAIP29/FCS/"
# Path of LAIP29 labeling
LAIP29_label_path = "<ENTER PATH>"
# Path of LAIP29 annotations
LAIP29_annotation_path = "<ENTER PATH>"

# OUTPUT FOLDERS
BLAST110_output = "output/BLAST110"
LAIP29_output = "output/LAIP29"
RBM18_output = "output/RBM18"

In [ ]:
# Define which markers we used for blast prediction
features = ["SSC-A_scaled", "Horizon V500-A", "PerCP-A", "PC7-A"]
# Markers used for reference models
markers = ["FITC-A", "PE-A", "PerCP-A", "PC7-A", "APC-A", 
           "APC-H7-A", "Horizon V450-A", "Horizon V500-A"]

# Reference data

In [ ]:
# Load the GMMclf
clf = joblib.load(BLAST110_output+"/GMMclf.pkl")

# Load the WBC regressor
reg = joblib.load(BLAST110_output+"/GMMclf_WBCreg.pkl")

ref_data = {"GMMclf":{}, "manual":{}}
for gating in ["GMMclf", "manual"]:
    for tube in ["P1", "P2", "P3", "P4"]:
        ref_data[gating][tube] = pd.read_csv(RBM18_output+"/RBM18_"+tube+"_blasts_"+gating+".csv", index_col=0)
        
# Load the reference models and quantiles
refGMMs = {"GMMclf":{}, "manual":{}}
percentiles = {"GMMclf":{}, "manual":{}}
for gating in ["GMMclf", "manual"]:
    for tube in ["P1", "P2", "P3", "P4"]:
        refGMMs[gating][tube] = joblib.load(RBM18_output+"/GMMref_"+tube+"_"+gating+".pkl")
        cutoffs = pd.read_csv(RBM18_output+"/GMMref_"+tube+"_"+gating+"_percentiles.csv", index_col=0)
        percentiles[gating][tube] = cutoffs.to_dict(orient='records')[0]

# Set up modeling

In [ ]:
nuSVMs = {"GMMclf":{}, "manual":{}}
for gating in ["GMMclf", "manual"]:
    for tube in ["P1", "P2", "P3", "P4"]:
        transform = Nystroem(gamma=2, random_state=42)
        clf_sgd = SGDOneClassSVM(nu=0.05, shuffle=True, fit_intercept=True, random_state=42)
        pipe_sgd = make_pipeline(transform, clf_sgd)
        nuSVMs[gating][tube] = pipe_sgd.fit(ref_data[gating][tube][markers]) 

In [ ]:
LOF = {"GMMclf":{}, "manual":{}}
for gating in ["GMMclf", "manual"]:
    for tube in ["P1", "P2", "P3", "P4"]:
        model = LocalOutlierFactor(novelty=True, contamination=0.05)
        LOF[gating][tube] = model.fit(ref_data[gating][tube][markers]) 

In [ ]:
def UMAP_HDBSCAN(test_blasts, ref_blasts):
    input_blasts = test_blasts.copy()
    if int(len(input_blasts) * 0.8) > len(ref_blasts):
        input_blasts = input_blasts.sample(len(ref_blasts), random_state=42)
        input_blasts["label"] = "test"
    else:
        ref_blasts = ref_blasts.sample(int(len(input_blasts) * 0.8), random_state=42)
        ref_blasts["label"] = "control"
    
    # Aggregate the data
    agg = pd.concat([input_blasts, ref_blasts])
    agg = shuffle(agg)

    # Get 3D UMAP coordinates
    reducer = umap.UMAP(min_dist=0, n_components=3, random_state=42, low_memory=False)
    reducer = reducer.fit(agg[markers])
    embedding = reducer.transform(agg[markers])
    embedding = pd.DataFrame(embedding)

    # Perform HDBSCAN clustering
    hdb = hdbscan.HDBSCAN(min_cluster_size=50, core_dist_n_jobs=1, prediction_data=True)
    hdb.fit(embedding.to_numpy())
    agg["HDBSCAN"] = hdb.labels_
    
    # Identify the anomalous cluster
    aberrant_dict = {-1: 0}
    for i in agg["HDBSCAN"].unique():
        if i == -1 or i == '-1':
            # Outliers were marked as control in original publication
            # Based on email correspondence
            aberrant_dict[i] = 0
        else:
            temp = agg[agg["HDBSCAN"]==i]
            frac = len(temp[temp["label"] == "control"]) / len(temp)
            if frac <= 0.05:
                aberrant_dict[i] = 1
            else:
                aberrant_dict[i] = 0
                
    # Identify anomalous cells in the original data
    embedding = reducer.transform(test_blasts[markers])
    embedding = pd.DataFrame(embedding)
    test_labels, _ = hdbscan.approximate_predict(hdb, embedding.to_numpy())      
    test_blasts["HDBSCAN"] = test_labels
    pred = [aberrant_dict[i] for i in test_blasts['HDBSCAN']]
    return pred

In [ ]:
def GMM_anomaly(test_blasts, ref_model, cutoff):
    test_blasts["LLR"] = ref_model.score_samples(test_blasts[markers])
    pred = np.where(test_blasts["LLR"] <= cutoff, 1, 0)
    return(pred)

In [ ]:
if not os.path.exists(LAIP29_output + "/benchmark/"):
    os.makedirs(LAIP29_output + "/benchmark/")

for root, dirs, files in os.walk(LAIP29_FCS_path):
    for file in files:      
        print(file)
        # Read and process FCS file
        sample_id = "_".join(file.split("_")[0:4])
        patient_id = "_".join(file.split("_")[0:2])
        timepoint = file.split("_")[2]
        tube = file.split('_')[3]
        ff = DataFrame.from_fcs(root+file)
        labels = pd.read_csv(LAIP29_label_path+sample_id+".csv", index_col=0, low_memory=False)
        ff = pd.merge(ff, labels)
        annotation = pd.read_csv(LAIP29_annotation_path+sample_id+".csv", index_col=0, low_memory=False)
        biggest_LAIP = list(annotation.sort_values("LAIP_WBC", ascending=False)["LAIP_WBC"])[0]
        if biggest_LAIP > 0.1:
            MRD_status = 1
        else:
            MRD_status = 0
        LAIP_events = len(ff[ff["LAIP"]==1])

        # Predict and save the WBC counts
        ff["NB_GMM_cluster"] = clf.gmm_class0.predict(ff[features])
        counts = pd.DataFrame(ff["NB_GMM_cluster"].value_counts())
        counts = counts.transpose().reset_index(drop=True)
        for i in range(0, clf.gmm_class0.n_components):
            if i not in counts:
                counts[i] = 0
        count_cols = [i for i in range(0, clf.gmm_class0.n_components)]
        WBC_count_pred = int(reg.predict(counts[count_cols]))
        
        # Determine manual MRD%
        WBC_count_manual = len(ff[ff["WBC"]==1])
        LAIP_perc = (LAIP_events / WBC_count_manual) * 100

        # Predict blasts
        ff["GMMclf"] = clf.predict(ff[features])
        for gating in ["GMMclf", "manual"]:
            if gating == "GMMclf":
                test_blasts = ff[ff[gating] == 1]
                other_cells = ff[ff[gating] == 0]
            else:
                test_blasts = ff[ff["Blast"] == 1]
                other_cells = ff[ff["Blast"] == 0]
            other_cells["pred"] = 0
            
            # nuSVM
            output_file = LAIP29_output + "/benchmark/" + sample_id + "_" + gating + "_nuSVM.csv"
            if not os.path.exists(output_file):
                test_blasts["pred"] = nuSVMs[gating][tube].predict(test_blasts[markers])
                pred = np.where(test_blasts["pred"] == -1, 1, 0)
                test_blasts["pred"] = pred
                concat = pd.concat([test_blasts, other_cells])
                if gating == "GMMclf":
                    cMRD_perc = (len(concat[concat["pred"] == 1]) / WBC_count_pred) * 100
                else:
                    cMRD_perc = (len(concat[concat["pred"] == 1]) / WBC_count_manual) * 100
                result = {"sample_id": sample_id,
                          "timepoint": timepoint,
                          "gating": gating,
                          "model": "nuSVM",
                          "gt_count": len(concat[concat["LAIP"]==1]),
                          "MRD": MRD_status,
                          "MRD%": biggest_LAIP,
                          "LAIP%": LAIP_perc,
                          "pred_count": len(concat[concat["pred"] == 1]),
                          "cMRD%": cMRD_perc,
                          "accuracy": accuracy_score(concat["LAIP"], concat["pred"]),
                          "precision": precision_score(concat["LAIP"], concat["pred"]),
                          "recall": recall_score(concat["LAIP"], concat["pred"]),
                          "f1": f1_score(concat["LAIP"], concat["pred"])}
                result = pd.DataFrame([result])
                result.to_csv(output_file)
                
            # LOF
            output_file = LAIP29_output + "/benchmark/" + sample_id + "_" + gating + "_LOF.csv"
            if not os.path.exists(output_file):
                test_blasts["pred"] = LOF[gating][tube].predict(test_blasts[markers])
                pred = np.where(test_blasts["pred"] == -1, 1, 0)
                test_blasts["pred"] = pred
                concat = pd.concat([test_blasts, other_cells])
                if gating == "GMMclf":
                    cMRD_perc = (len(concat[concat["pred"] == 1]) / WBC_count_pred) * 100
                else:
                    cMRD_perc = (len(concat[concat["pred"] == 1]) / WBC_count_manual) * 100
                result = {"sample_id": sample_id,
                          "timepoint": timepoint,
                          "gating": gating,
                          "model": "LOF",
                          "gt_count": len(concat[concat["LAIP"]==1]),
                          "MRD": MRD_status,
                          "MRD%": biggest_LAIP,
                          "LAIP%": LAIP_perc,
                          "pred_count": len(concat[concat["pred"] == 1]),
                          "cMRD%": cMRD_perc,
                          "accuracy": accuracy_score(concat["LAIP"], concat["pred"]),
                          "precision": precision_score(concat["LAIP"], concat["pred"]),
                          "recall": recall_score(concat["LAIP"], concat["pred"]),
                          "f1": f1_score(concat["LAIP"], concat["pred"])}
                result = pd.DataFrame([result])
                result.to_csv(output_file)
                
            # UMAP-HDBSCAN
            output_file = LAIP29_output + "/benchmark/" + sample_id + "_" + gating + "_UMAP-HDBSCAN.csv"
            if not os.path.exists(output_file):
                # Get the reference blasts
                ref_blasts = ref_data[gating][tube]
                test_blasts["pred"] = UMAP_HDBSCAN(test_blasts, ref_blasts)
                concat = pd.concat([test_blasts, other_cells])
                if gating == "GMMclf":
                    cMRD_perc = (len(concat[concat["pred"] == 1]) / WBC_count_pred) * 100
                else:
                    cMRD_perc = (len(concat[concat["pred"] == 1]) / WBC_count_manual) * 100
                result = {"sample_id": sample_id,
                          "timepoint": timepoint,
                          "gating": gating,
                          "model": "UMAP-HDBSCAN",
                          "gt_count": len(concat[concat["LAIP"]==1]),
                          "MRD": MRD_status,
                          "MRD%": biggest_LAIP,
                          "LAIP%": LAIP_perc,
                          "pred_count": len(concat[concat["pred"] == 1]),
                          "cMRD%": cMRD_perc,
                          "accuracy": accuracy_score(concat["LAIP"], concat["pred"]),
                          "precision": precision_score(concat["LAIP"], concat["pred"]),
                          "recall": recall_score(concat["LAIP"], concat["pred"]),
                          "f1": f1_score(concat["LAIP"], concat["pred"])}
                result = pd.DataFrame([result])
                result.to_csv(output_file)
                
            # GMM
            output_file = LAIP29_output + "/benchmark/" + sample_id + "_" + gating + "_GMM.csv"
            if not os.path.exists(output_file):
                ref_model = refGMMs[gating][tube]
                results = []
                for percentile in percentiles[gating][tube]:
                    cutoff = percentiles[gating][tube][percentile]
                    test_blasts["pred"] = GMM_anomaly(test_blasts, ref_model, cutoff)
                    concat = pd.concat([test_blasts, other_cells])
                    if gating == "GMMclf":
                        cMRD_perc = (len(concat[concat["pred"] == 1]) / WBC_count_pred) * 100
                    else:
                        cMRD_perc = (len(concat[concat["pred"] == 1]) / WBC_count_manual) * 100
                    result = {"sample_id": sample_id,
                              "timepoint": timepoint,
                              "gating": gating,
                              "model": "GMM",
                              "percentile": percentile,
                              "gt_count": len(concat[concat["LAIP"]==1]),
                              "MRD": MRD_status,
                              "MRD%": biggest_LAIP,
                              "LAIP%": LAIP_perc,
                              "pred_count": len(concat[concat["pred"] == 1]),
                              "cMRD%": cMRD_perc,
                              "accuracy": accuracy_score(concat["LAIP"], concat["pred"]),
                              "precision": precision_score(concat["LAIP"], concat["pred"]),
                              "recall": recall_score(concat["LAIP"], concat["pred"]),
                              "f1": f1_score(concat["LAIP"], concat["pred"])}
                    results.append(result)
                result = pd.DataFrame(results)
                result.to_csv(output_file)